In [ ]:
# creating the tRNA network for enzyme sweeps
#completed 1/31/17

#this code calls a recursive function that loops through the tRNA network, removing downstream rxns from the network 
#once an upstream reaction is removed (due to an enzyme knockout). it saves the network map (dictionary) as a .json file.
#These .json networks will have to be inputted into the tRNA workflow to recreate tRNA_charging.py and post_transcriptional_files.txt



In [3]:
# python imports
import re
import json
from os.path import join
import cPickle
import numpy as np
import math
import itertools

# third party imports\
import pandas
import escher
import cloudpickle

# ecoli me
import ecolime
from ecolime.flat_files import *
from ecolime.ecoli_k12 import *
from ecolime import (ribosome, tRNA_charging, transcription, translocation, 
                     chaperones, DNA_replication, translation)
from ecolime.characterization.biomass_allocation import get_membrane_composition, get_membrane_protein

from cobrame import *
from cobrame.util import building, dogma
from cobrame.util.mass import compute_RNA_mass
from cobrame.solve.algorithms import binary_search, fva, solve_at_growth_rate
from cobrame.solve.symbolic import compile_expressions

from cloudpickle import load
import cPickle

from cobrame import ComplexData, TranscribedGene, ModificationData
from cobrame.util.building import add_modification_data

import cobra
import os
from tqdm import tqdm
import copy
from copy import deepcopy

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [4]:
def find_trna_enzymes():
    """
    finds all enzymes used in tRNA modifications
    """
    results_list = []
    for tRNA_modification in tRNA_charging.trna_modification:
        tRNA_enzyme_list = tRNA_charging.trna_modification[tRNA_modification]['machines'] 
        
        #if 'YhhP_mono' in tRNA_enzyme_list:
            #print tRNA_modification, 'YhhP_mono'
        
        for tRNA_enzyme in tRNA_enzyme_list:
            if tRNA_enzyme not in results_list and tRNA_enzyme != None :
                results_list.append(tRNA_enzyme)
    return results_list

# tRNA recursion negative sweep with QCQA for positions works!


In [5]:
#recursive loop #### LATEST ATTEMPT IT WORKSSSS!!!!!!! 2/2/17

def tRNA_network_map_enzyme_sweep(deleted_modification_list, position_list, graph_input):
    """
    removes modifications from the tRNA network map when specific enzymes are knocked out!
    
    """
    import copy
    from copy import deepcopy
    #print 'entered'
    graph_copy = copy.deepcopy(graph_input)
    proceed_to_QCQA = False
    #recursive end case
    print '1: recurision input list --> ' ,deleted_modification_list
    if deleted_modification_list == []:
        proceed_to_QCQA = True
    
    #recursion: identify downstream modifications that must be removed once an enzyme (upstream rxn ) is knocked out
    else:
        new_deleted_modification_list = []
        for base in graph_input.keys():
            for downstream_mod in graph_input[base].keys():
                for deleted_modification in deleted_modification_list:
                    if deleted_modification in graph_input[base][downstream_mod]['upstream_rxns']:
                        if len(graph_input[base][downstream_mod]['enzymes'].keys()) > 1:
                            print 'error'
                        else:     
                            graph_copy[base][downstream_mod]['upstream_rxns'].remove(deleted_modification)

                if graph_copy[base][downstream_mod]['upstream_rxns'] == []:
                    del graph_copy[base][downstream_mod]

                    new_deleted_modification_list.append(downstream_mod)
        #returns the recursive function w/new modifications to be deleted from the map
        print '2: recursion new list --> ', new_deleted_modification_list
        return tRNA_network_map_enzyme_sweep(new_deleted_modification_list, position_list, graph_copy)
    
    print "Proceed to QCQA?", graph_input == graph_copy
    network_size = 0 
    for base in graph_copy:
        network_size = network_size + len(graph_copy[base].keys())
    
    if proceed_to_QCQA == True:
        qcqa_delete_list, qcqa_network = QCQA_network_checker(graph_copy)
        if qcqa_delete_list == []:
            print '3: QCQA satisified'
            print 'network size = %i' % network_size
            #print 'qcqa list = []'
            return qcqa_network
        else:
            #print qcqa_network == graph_input
            return tRNA_network_map_enzyme_sweep(qcqa_delete_list, position_list, qcqa_network)

In [6]:

def QCQA_network_checker(network):

    qcqa_base_dict = {'Uridine_mods_paths' : 'U',
                     'Guanine_mods_paths' : 'G',
                      'Adenine_mods_paths' : 'A',
                      'Cytidine_mods_paths' : 'C'}
    import copy
    from copy import deepcopy
    network_copy = copy.deepcopy(network)
    qcqa_delete_list = []
    
    for base in network.keys():
        nucleotide = qcqa_base_dict[base]
        for modification in network[base].keys():

            upstream_rxn_list =  network[base][modification]['upstream_rxns']
            modification_position_list = network[base][modification]['enzymes'].keys()
            if upstream_rxn_list != [nucleotide]:
                #print '\n'
                #print modification, upstream_rxn_list
                #print modification_position_list
                upstream_position_list = []
                for upstream_rxn in upstream_rxn_list:
                    upstream_positions = network[base][upstream_rxn]['enzymes'].keys()
                    upstream_position_list = upstream_position_list + upstream_positions

                for position in modification_position_list:
                    if position not in set(upstream_position_list):
                        if modification not in qcqa_delete_list:
                            qcqa_delete_list.append(modification)
    
    
    for base in network.keys():
        for modification in network[base].keys():
            if modification in qcqa_delete_list:
                del network_copy[base][modification]
                print 'This modification was deleted by QCQA because upstream position disagreement: %s' %(modification)
    return qcqa_delete_list, network_copy



In [7]:
# 'additional_enzymes': {'YccK_mono': 1, 'YheLMN_cplx': 1, 'YhhP_mono': 1} TrmU_mono for ->s2U additions ????
# these for enzymes are important for s2U additions
# they are not included in the tRNA network map, thus they dont affect this network change.
#but they ARE included in the tRNA reaction details from MODS_URIDINE_2.py
#WHAT TO DO????

In [211]:
from tRNA_MODOMICS_network_map import * #adenine_mods_paths, uridine_mods_paths, guanine_mods_paths, cytidine_mods_paths

#cycle through tRNA_enzymes
for tRNA_enzyme in tqdm(find_trna_enzymes()):
    print '\n', tRNA_enzyme
    deleted_modification_list = []
    position_list= []
    network_map = {}
    network_map.update({'Uridine_mods_paths' :uridine_mods_paths })
    network_map.update({'Adenine_mods_paths' :adenine_mods_paths })
    network_map.update({'Cytidine_mods_paths' :cytidine_mods_paths })
    network_map.update({'Guanine_mods_paths' :guanine_mods_paths })
    network_map_copy = copy.deepcopy(network_map)
    #creating the "deleted modification list"
    for base in network_map.keys():
        for modification in network_map[base].keys():
            for position, enzyme in network_map[base][modification]['enzymes'].items():
                if enzyme[0] == tRNA_enzyme:
                    print '%s at position %s' %(modification, position)
                    del network_map_copy[base][modification]['enzymes'][position]
                    if position not in position_list:
                        position_list.append(position)

                    if network_map_copy[base][modification]['enzymes'] == {}:
                        del network_map_copy[base][modification]
                        deleted_modification_list.append(modification)
    #creating the network via recursive function
    network_output = tRNA_network_map_enzyme_sweep(deleted_modification_list, position_list, network_map_copy)
    
    #saving the network
    save_file_name = str(tRNA_enzyme) + '_removed_from_' + 'tRNA_MODOMICS_network_map.json'      
    path = './modified_tRNA_modomics_network_map_for_sweeps/' 
    with open(path + save_file_name, 'w') as f:
            json.dump(network_output, f)
    

  0%|          | 0/37 [00:00<?, ?it/s]


TmcA_mono
C_to_ac4C at position 34
1: recurision input list -->  ['C_to_ac4C']
2: recursion new list -->  []
1: recurision input list -->  []
Proceed to QCQA? True
3: QCQA satisified
network size = 62

RlmN_mono_mod_1:4fe4s
A_to_m2A at position 37
A_to_m2A at position 2503
1: recurision input list -->  ['A_to_m2A']
2: recursion new list -->  []
1: recurision input list -->  []
Proceed to QCQA? True
3: QCQA satisified
network size = 62

TruB_mono
U_to_Y at position 55
1: recurision input list -->  []
Proceed to QCQA? True
3: QCQA satisified
network size = 63

MnmEG_cplx_mod_fad_mod_2:k
se2U_to_nm5se2U at position 34
U_to_nm5U at position 34
U_to_cmnm5U at position 34
s2U_to_nm5s2U at position 34
s2U_to_cmnm5s2U at position 34
Um_to_cmnm5Um at position 34
se2U_to_cmnm5se2U at position 34
1: recurision input list -->  ['se2U_to_nm5se2U', 'U_to_nm5U', 'U_to_cmnm5U', 's2U_to_nm5s2U', 's2U_to_cmnm5s2U', 'Um_to_cmnm5Um', 'se2U_to_cmnm5se2U']
2: recursion new list -->  ['cmnm5U_to_cmnm5Um', '

100%|██████████| 37/37 [00:00<00:00, 326.26it/s]

 []
Proceed to QCQA? True
3: QCQA satisified
network size = 63

YfiC_mono
A_to_m6A at position 37
1: recurision input list -->  []
Proceed to QCQA? True
3: QCQA satisified
network size = 63

TilS_mono
C_to_k2C at position 34
1: recurision input list -->  ['C_to_k2C']
2: recursion new list -->  []
1: recurision input list -->  []
Proceed to QCQA? True
3: QCQA satisified
network size = 62

RluA_mono
U_to_Y at position 32
U_to_Y at position 746
1: recurision input list -->  []
Proceed to QCQA? True
3: QCQA satisified
network size = 63


In [8]:
#the CONTROL tRNA_MODOMICS_network_map
from tRNA_MODOMICS_network_map import * #adenine_mods_paths, uridine_mods_paths, guanine_mods_paths, cytidine_mods_paths

network_map = {}
network_map.update({'Uridine_mods_paths' :uridine_mods_paths })
network_map.update({'Adenine_mods_paths' :adenine_mods_paths })
network_map.update({'Cytidine_mods_paths' :cytidine_mods_paths })
network_map.update({'Guanine_mods_paths' :guanine_mods_paths })
network_size = 0 
for base in network_map:
    network_size = network_size + len(network_map[base].keys())
print 'network size = %i' % network_size
#saving the network
save_file_name = 'control_' + 'tRNA_MODOMICS_network_map.json'      
path = './modified_tRNA_modomics_network_map_for_sweeps/' 
with open(path + save_file_name, 'w') as f:
        json.dump(network_map, f)


network size = 63


# trna positive enzyme sweep

In [22]:
#positive sweep
from tRNA_MODOMICS_network_map import * #adenine_mods_paths, uridine_mods_paths, guanine_mods_paths, cytidine_mods_paths

#cycle through tRNA_enzymes
for tRNA_enzyme in tqdm(find_trna_enzymes()):
    print '\n', tRNA_enzyme
    deleted_modification_list = []
    position_list= []
    network_map = {}
    network_map.update({'Uridine_mods_paths' :uridine_mods_paths })
    network_map.update({'Adenine_mods_paths' :adenine_mods_paths })
    network_map.update({'Cytidine_mods_paths' :cytidine_mods_paths })
    network_map.update({'Guanine_mods_paths' :guanine_mods_paths })
    network_map_copy = copy.deepcopy(network_map)
    #creating the "deleted modification list"
    for base in network_map.keys():
        for modification in network_map[base].keys():
            for position, enzyme in network_map[base][modification]['enzymes'].items():
                #here we delete enzymes that are not the one of interest ( positive sweep)
                if enzyme[0] != tRNA_enzyme:
                    del network_map_copy[base][modification]['enzymes'][position]
                    if position not in position_list:
                        position_list.append(position)

                    if network_map_copy[base][modification]['enzymes'] == {}:
                        del network_map_copy[base][modification]
                        deleted_modification_list.append(modification)
                        
                else:
                    print modification
    network_output = tRNA_network_map_enzyme_sweep(deleted_modification_list, position_list, network_map_copy)
    
    #saving the network
    save_file_name = str(tRNA_enzyme) + '_positive_ONLY_' + 'tRNA_MODOMICS_network_map.json'      
    path = './modified_tRNA_modomics_network_map_for_sweeps/positive_enzyme_sweeps/' 
    with open(path + save_file_name, 'w') as f:
            json.dump(network_output, f)
    

100%|██████████| 37/37 [00:00<00:00, 1212.16it/s]


TmcA_mono
C_to_ac4C
1: recurision input list -->  ['QtRNA_to_gluQtRNA', 'G_to_m1G', 'G_to_Gm', 'G_to_m2G', 'oQtRNA_to_QtRNA', 'preq1tRNA_to_oQtRNA', 'G_to_preq1tRNA', 'G_to_m7G', 'C_to_s2C', 'C_to_k2C', 'm4Cm_to_m44Cm', 'C_to_m5C', 'm4C_to_m44C', 'C_to_m4C', 'C_to_Cm', 'Cm_to_m4Cm', 'm4C_to_m4Cm', 'A_to_m2A', 'i6A_to_ms2i6A', 't6A_to_m6t6A', 'm6A_to_m66A', 'A_to_m1A', 'A_to_i6A', 'A_to_I', 'A_to_m6A', 'A_to_t6A', 't6A_to_ct6A', 'ho5U_to_mo5U', 'se2U_to_nm5se2U', 'cmnm5s2U_to_cmnm5se2U', 'U_to_acp3U', 'U_to_nm5U', 'mnm5s2U_to_mnm5ges2U', 'U_to_cmnm5U', 's2U_to_se2U', 'nm5s2U_to_mnm5s2U', 'U_to_s2U', 's2U_to_ges2U', 'cmnm5U_to_cmnm5s2U', 'cmnm5se2U_to_nm5se2U', 's2U_to_nm5s2U', 'ho5U_to_cmo5U', 'U_to_m3U', 'mnm5s2U_to_mnm5se2U', 's2U_to_cmnm5s2U', 'U_to_ho5U', 'U_to_Um', 'nm5U_to_mnm5U', 'U_to_m5U', 'mnm5U_to_mnm5s2U', 'cmnm5U_to_cmnm5Um', 'U_to_DDusgen', 'U_to_Y', 'Um_to_cmnm5Um', 'nm5s2U_to_nm5se2U', 'cmnm5s2U_to_nm5s2U', 'U_to_s4U', 'U_to_DDusA', 'nm5se2U_to_mnm5se2U', 'cmnm5U_to_nm5

In [9]:
#control for positive sweeps
#the CONTROL tRNA_MODOMICS_network_map


network_map = {}
network_map.update({'Uridine_mods_paths' :{} })
network_map.update({'Adenine_mods_paths' :{} })
network_map.update({'Cytidine_mods_paths' :{} })
network_map.update({'Guanine_mods_paths' :{} })
network_size = 0 
for base in network_map:
    network_size = network_size + len(network_map[base].keys())
print 'network size = %i' % network_size
#saving the network
save_file_name = 'control_' + 'tRNA_MODOMICS_network_map.json'      
path = './modified_tRNA_modomics_network_map_for_sweeps/positive_enzyme_sweeps/' 
with open(path + save_file_name, 'w') as f:
        json.dump(network_map, f)


network size = 0


# trouble shooting
